### Please install the required Python modules/SDKs

In [ ]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

In [ ]:
!pip install Pillow==8.4

In [ ]:
import io
import datetime
import pandas as pd
from PIL import Image
import requests
import io
import glob, os, sys, time, uuid

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw

from video_indexer import VideoIndexer
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import TrainingStatusType
from msrest.authentication import CognitiveServicesCredentials

In [ ]:
# Todo: add resources
CONFIG = {
    'SUBSCRIPTION_KEY': 'YOUR VIDEO INDEXER SUBSCRIPTION KEY',
    'LOCATION': 'trial',
    'ACCOUNT_ID': 'YOUR VIDEO INDEXER ACCOUNT ID'
}

video_analysis = VideoIndexer(
    vi_subscription_key=CONFIG['SUBSCRIPTION_KEY'],
    vi_location=CONFIG['LOCATION'],
    vi_account_id=CONFIG['ACCOUNT_ID']
)

In [ ]:
video_analysis.check_access_token()

### TODO: Processing RAW Json

In [ ]:
# Todo: add the video ID of your uploaded video
video_id = 'ENTER YOUR VIDEO ID'

In [ ]:
# Get the raw Json info
video_analysis.get_video_info(video_id)
info = video_analysis.get_video_info(video_id, video_language='English')

In [ ]:
if len(info['videos'][0]['insights']['faces'][0]['thumbnails']):
    print("We found {} faces in this video.".format(str(len(info['videos'][0]['insights']['faces'][0]['thumbnails']))))

### TODO: Extract And Save The Face-Specific Thumbnails 

In [ ]:
# Todo: get Thumbnail ID from the Analysis JSON 
images = []
img_raw = []
img_strs = []
for each_thumb in info['videos'][0]['insights']['faces'][0]['thumbnails']:
    if 'fileName' in each_thumb and 'id' in each_thumb:
        file_name = each_thumb['fileName']
        thumb_id = each_thumb['id']
        # Todo: get the image code for the thumbnail ID 
        # using the video_analysis.get_thumbnail_from_video_indexer object
        img_code = None
        img_strs.append(img_code)
        img_stream = io.BytesIO(img_code)
        img_raw.append(img_stream)
        img = Image.open(img_stream)
        images.append(img)

In [ ]:
# Plot the images
for img in images:
    print(img.info)
    plt.figure()
    plt.imshow(img)

In [ ]:
# Save the images
i = 1
for img in images:
    print(type(img))
    img.save('video-analyzer-face' + str(i) + '.jpg')
    i= i+ 1

In [ ]:
# Verify the download process 
!ls video-analyzer-face*.jpg

### TODO: View And Convert Thumbnail To Visible Image

In [ ]:
info['videos'][0]['insights']['faces'][0]['thumbnails']

In [ ]:
# Todo: enter one of the thumbnail output you got from the previous cell
thumbnail_id='ENTER THUMBNAIL HERE'

In [ ]:
# Todo: convert encoded image to visible image 
# using the video_analysis.get_thumbnail_from_video_indexer object
img_code = None
img_stream = io.BytesIO(img_code)
img = Image.open(img_stream)
imshow(img)

### TODO: Collecting Faces From Video Analyzer

In [ ]:
# Todo: add resources
AVKASH_FACE_KEY = "YOUR FACE SERVICE KEY"
AVKASH_FACE_ENDPOINT = "YOUR FACE SERVICE ENDPOINT"

# Create a client
face_client = FaceClient(AVKASH_FACE_ENDPOINT, CognitiveServicesCredentials(AVKASH_FACE_KEY))
face_client.api_version

### Build the Person Model Using Person Images 
**Note: here you need to build a person model using the person images from the previous exercise. Before moving forward, make sure you upload all the images to the workspace.**

Majority of the code is provided. But you may need to modify image names and the person group name.

In [ ]:
!ls human-face*.jpg

In [ ]:
my_face_images = [file for file in glob.glob('*.jpg') if file.startswith("human-face")]
print(my_face_images)

In [ ]:
for img in my_face_images:
    with open(img, 'rb') as img_code:
        img_view_ready = Image.open(img_code)
        plt.figure()
        plt.imshow(img_view_ready)

In [ ]:
# Note if this UUID already used earlier, you will get an error 
# Replace "person-avkash" with your own PersonModel name
PERSON_GROUP_ID = str(uuid.uuid4())
person_group_name = 'person-avkash'

In [ ]:
## This code is taken from Azure face SDK 
## ---------------------------------------
def build_person_group(client, person_group_id, pgp_name):
    print('Create and build a person group...')
    # Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
    print('Person group ID:', person_group_id)
    client.person_group.create(person_group_id = person_group_id, name=person_group_id)

    # Create a person group person.
    my_face = client.person_group_person.create(person_group_id, pgp_name)
    # Find all jpeg images of human in working directory.
    my_face_images = [file for file in glob.glob('*.jpg') if file.startswith("human-face")]
    # Add images to a Person object
    for image_p in my_face_images:
        with open(image_p, 'rb') as w:
            client.person_group_person.add_face_from_stream(person_group_id, my_face.person_id, w)

    # Train the person group, after a Person object with many images were added to it.
    client.person_group.train(person_group_id)

    # Wait for training to finish.
    while (True):
        training_status = client.person_group.get_training_status(person_group_id)
        print("Training status: {}.".format(training_status.status))
        if (training_status.status is TrainingStatusType.succeeded):
            break
        elif (training_status.status is TrainingStatusType.failed):
            client.person_group.delete(person_group_id=PERSON_GROUP_ID)
            sys.exit('Training the person group has failed.')
        time.sleep(5)

In [ ]:
build_person_group(face_client, PERSON_GROUP_ID, person_group_name)

In [ ]:
# Detect all faces in query image list, then add their face IDs to a new list.
def detect_faces(client, query_images_list):
    print('Detecting faces in query images list...')

    face_ids = {} # Keep track of the image ID and the related image in a dictionary
    for image_name in query_images_list:
        image = open(image_name, 'rb') # BufferedReader
        print("Opening image: ", image.name)
        time.sleep(5)

        # Detect the faces in the query images list one at a time, returns list[DetectedFace]
        faces = client.face.detect_with_stream(image)  

        # Add all detected face IDs to a list
        for face in faces:
            print('Face ID', face.face_id, 'found in image', os.path.splitext(image.name)[0]+'.jpg')
            # Add the ID to a dictionary with image name as a key.
            # This assumes there is only one face per image (since you can't have duplicate keys)
            face_ids[image.name] = face.face_id

    return face_ids

In [ ]:
ids = detect_faces(face_client, my_face_images)

In [ ]:
ids

### TODO: Matching Face From The Person Model With Face From Video Analyzer 

In [ ]:
# Todo: choose a image from the Video Analyzer and change the local path to read the image
dl_image = open('/home/workspace/YOUR-FILE-NAME.jpg', 'rb')

# Detect faces in that image
dl_faces = face_client.face.detect_with_stream(dl_image)

In [ ]:
# View Face ID and then save it into the list of already saved Face IDs
for face in dl_faces:
    print('Face ID', face.face_id, 'found in image', dl_image)
    # Add the ID to a dictionary with image name as a key.
    # This assumes there is only one face per image (since you can't have duplicate keys)
    ids['video-analyzer.png'] = face.face_id

In [ ]:
# Now, you should have n + 1 Face IDs in the list
ids

In [ ]:
# Todo: enter the video analyzer face ID of from the output of the cell above
get_the_face_id_from_the_video_analyzer = 'ENTER FACE ID HERE'

In [ ]:
# Identify the face from the video analyzer
person_gp_results = face_client.face.identify([get_the_face_id_from_the_video_analyzer], PERSON_GROUP_ID)

In [ ]:
for result in person_gp_results:
    if result.candidates:
        for candidate in result.candidates:
            print("The Identity match confidence is {}".format(candidate.confidence))
    else:
        print("Can't verify the identity with the person group")